In [1]:
import json
import pprint

In [2]:
import z3
x=z3.Int('x')
y=z3.Int('y')
constraints=[z3.And((x<2),(y-x)<1)]

In [3]:
optimizer = z3.Optimize()
optimizer.add(constraints)
optimizer.minimize(x+y)
print(optimizer.check())
print(optimizer.model())

sat
[y = -1, x = 0]


In [4]:
optimizer = z3.Optimize()
optimizer.add(constraints)
optimizer.maximize(x+y)
print(optimizer.check())
print(optimizer.model())

sat
[y = 1, x = 1]


In [5]:
with open ('dummy_km.json') as file:
    data = json.load(file)
    
#pprint.pprint(data)

In [6]:
data['VG1']['dependencies']['nonlocal']

['APP2', 'APP3']

I think there needs to be a list of all resource types. I'm not sure how to guarantee the ordering otherwise. 

In [7]:
[j for i in data for j in data[i]['resources'] ]#depth first, pull all elements 

['memory',
 'storage',
 'memory',
 'storage',
 'memory',
 'storage',
 'memory',
 'storage',
 'memory',
 'storage']

In [8]:
resourceTypes = set([j for i in data for j in data[i]['resources'] ])
print(resourceTypes)

{'memory', 'storage'}


nodeList

In [9]:
nodeList = list()
for i in data:
    if data[i]['type']=='node':
        nodeList.append(i)
print (nodeList)

['FSSN3', 'FSSN2', 'FSSN1']


In [10]:
appList = list()
for i in data:
    if data[i]['type']=='app':
        appList.append(i)
print (appList)

['VG1', 'APP2']


In [11]:
a2n = [ [ z3.Int("a2n_%s_%s" % (i, j)) for j in range(len(nodeList)) ]
            for i in range(len(appList)) ]
print(a2n)

[[a2n_0_0, a2n_0_1, a2n_0_2], [a2n_1_0, a2n_1_1, a2n_1_2]]


In [12]:
a2n = [ [ z3.Int("a2n_%son%s" % (i, j)) for j in nodeList ]
            for i in appList]
print(a2n)

[[a2n_VG1onFSSN3, a2n_VG1onFSSN2, a2n_VG1onFSSN1], [a2n_APP2onFSSN3, a2n_APP2onFSSN2, a2n_APP2onFSSN1]]


matrix for resource x node matrix

In [13]:
#build and check format for matrix of node resources
rsrc_per_node = [[z3.Int("rpn_%s_%s" %(k, j)) for j in nodeList] for k in resourceTypes]
rpn= [[data[j]['resources'][k] for j in nodeList] for k in resourceTypes]

k = 1
j = 0
pprint.pprint (rsrc_per_node)
pprint.pprint(rpn)
print("index [{}][{}] {} {}".format(k,j, rsrc_per_node[k][j], rpn[k][j]))

[[rpn_memory_FSSN3, rpn_memory_FSSN2, rpn_memory_FSSN1],
 [rpn_storage_FSSN3, rpn_storage_FSSN2, rpn_storage_FSSN1]]
[[512, 256, 1512], [1024, 512, 1024]]
index [1][0] rpn_storage_FSSN3 1024


In [14]:
#build and check format for matrix of app resource requirements
print ([i for i in appList])
rsrc_per_app = [[z3.Int("rpa_%s_%s" %(k, i)) for i in appList] for k in resourceTypes]

k = 1
i = 1

pprint.pprint(rsrc_per_app)
rpa= [[data[i]['resources'][k] for i in appList] for k in resourceTypes]
print("index [{}][{}] {} {}".format(k,i, rsrc_per_app[k][i], rpa[k][i]))

['VG1', 'APP2']
[[rpa_memory_VG1, rpa_memory_APP2], [rpa_storage_VG1, rpa_storage_APP2]]
index [1][1] rpa_storage_APP2 512


In [15]:
#make sure resources match when k changes. 

k = 1
i = 0
j = 1
print("index [{}][{}] {} {}".format(k,j, rsrc_per_node[k][j], rpn[k][j]))
print("index [{}][{}] {} {}".format(k,i, rsrc_per_app[k][i], rpa[k][i]))

rpn[k][j] >= rpa[k][i]

index [1][1] rpn_storage_FSSN2 512
index [1][0] rpa_storage_VG1 512


True

In [23]:
val_a2n = [ z3.Or(a2n[i][j]==0,a2n[i][j]==1,a2n[i][j]==2) for j in range(len(nodeList))
          for i in range(len(appList)) ]
print(val_a2n)

[Or(a2n_VG1onFSSN3 == 0,
   a2n_VG1onFSSN3 == 1,
   a2n_VG1onFSSN3 == 2), Or(a2n_APP2onFSSN3 == 0,
   a2n_APP2onFSSN3 == 1,
   a2n_APP2onFSSN3 == 2), Or(a2n_VG1onFSSN2 == 0,
   a2n_VG1onFSSN2 == 1,
   a2n_VG1onFSSN2 == 2), Or(a2n_APP2onFSSN2 == 0,
   a2n_APP2onFSSN2 == 1,
   a2n_APP2onFSSN2 == 2), Or(a2n_VG1onFSSN1 == 0,
   a2n_VG1onFSSN1 == 1,
   a2n_VG1onFSSN1 == 2), Or(a2n_APP2onFSSN1 == 0,
   a2n_APP2onFSSN1 == 1,
   a2n_APP2onFSSN1 == 2)]


In [24]:
rsrc_constraint = [rpn[k][j] >= z3.Sum([a2n[i][j]*rpa[k][i] for i in range(len(appList))]) 
 for j in range(len(nodeList))
 for k in range(len(resourceTypes))]

In [25]:
solver = z3.Solver()
solver.add(val_a2n + rsrc_constraint)

In [19]:
solver.check()

sat

In [20]:
solver.model()

[a2n_VG1onFSSN3 = 0,
 a2n_VG1onFSSN2 = 0,
 a2n_VG1onFSSN1 = 0,
 a2n_APP2onFSSN1 = 0,
 a2n_APP2onFSSN3 = 0,
 a2n_APP2onFSSN2 = 0]

In [21]:
a2n

[[a2n_VG1onFSSN3, a2n_VG1onFSSN2, a2n_VG1onFSSN1],
 [a2n_APP2onFSSN3, a2n_APP2onFSSN2, a2n_APP2onFSSN1]]

In [22]:
from functools import reduce
optimizer = z3.Optimize()
constraints = val_a2n + rsrc_constraint
optimizer.add(constraints)
optimizer.maximize(reduce(lambda x, y: x + sum(y), a2n, 0))
print(optimizer.check())
print(optimizer.model())

sat
[a2n_VG1onFSSN3 = 1,
 a2n_APP2onFSSN3 = 1,
 a2n_VG1onFSSN2 = 0,
 a2n_APP2onFSSN2 = 1,
 a2n_VG1onFSSN1 = 1,
 a2n_APP2onFSSN1 = 1]
